In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Oct  7 08:11:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.2 MB/s 
     |████████████████████████████████| 6.6 MB 59.3 MB/s 
     |████████████████████████████████| 163 kB 69.3 MB/s 


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mental Health/Mental-health-related-subreddits-modified.csv")
df.head()

,Content,Category,label
0,guys the civil war movies and games. keep givi...,schizophrenia,0
1,"What do I have? Hi everyone, I just found this...",anxiety,1
2,Buster from Arthur... A possibly random and we...,schizophrenia,0
3,anxiety attack at the movies I took about 50mg...,anxiety,1
4,Today’s reflections on depression I don’t know...,BPD,2


In [ ]:
df['Category'].value_counts()

anxiety          86132
depression       85994
bipolar          41052
BPD              38137
schizophrenia    17459
autism            7132
Name: Category, dtype: int64

In [ ]:
df.shape

(275906, 3)

In [ ]:
x = len(df)
x = int((x/100) * 80) # dividing to consider 80% data
print(x)

220724


In [ ]:
df=df.sample(x)

In [ ]:
df.shape

(220724, 3)

In [ ]:
df.Category.value_counts()

anxiety          69031
depression       68639
bipolar          32884
BPD              30512
schizophrenia    13954
autism            5704
Name: Category, dtype: int64

In [ ]:
possible_labels = ['schizophrenia', 'anxiety', 'BPD', 'depression', 'bipolar',
       'autism']

In [ ]:
#possible_labels = df.Category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'schizophrenia': 0,
 'anxiety': 1,
 'BPD': 2,
 'depression': 3,
 'bipolar': 4,
 'autism': 5}

In [ ]:
df.head()

,Content,Category,label
123432,It's never gonna change. Many things feel like...,depression,3
166197,Wave of euphoria at night time? I'll be depres...,depression,3
263947,Using dating apps when recovering from crippli...,anxiety,1
56752,What to do about pent up anger If somebody doe...,anxiety,1
187899,Do you ever secretly hope plans fall though so...,anxiety,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['Category', 'label', 'data_type']).count()

Content
Category      label data_type         
BPD           2     train        24410
                    val           6102
anxiety       1     train        55225
                    val          13806
autism        5     train         4563
                    val           1141
bipolar       4     train        26307
                    val           6577
depression    3     train        54911
                    val          13728
schizophrenia 0     train        11163
                    val           2791

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Content.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256,   #256 default 
    truncation=True,  #added by me
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Content.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256,   #256 default
    truncation=True,    #added by me 
    return_tensors='pt'
)




/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
torch.save(encoded_data_train, '/content/drive/MyDrive/Colab Notebooks/Mental Health/final_encoded_data_train_80%.pt')

In [ ]:
torch.save(encoded_data_val, '/content/drive/MyDrive/Colab Notebooks/Mental Health/final_encoded_data_val_80%.pt')

**Preparing Tensor**

In [ ]:
encoded_data_train = torch.load('/content/drive/MyDrive/Colab Notebooks/Mental Health/final_encoded_data_train_80%.pt')

In [ ]:
encoded_data_val = torch.load('/content/drive/MyDrive/Colab Notebooks/Mental Health/final_encoded_data_val_80%.pt')

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(176579, 44145)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 50 # higher batch_size caused RAM crash 

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/finetuned_BERT_epoch_{epoch}.model')
    
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/3532 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.6330418382198597
Validation loss: 0.5307158774792271
F1 Score (Weighted): 0.8181690132217428


Epoch 2:   0%|          | 0/3532 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.49433882522691014
Validation loss: 0.5198642909526825
F1 Score (Weighted): 0.821938406854573


...


Epoch 8
Training loss: 0.1943972006715446
Validation loss: 0.7145068768669228
F1 Score (Weighted): 0.8125300109355298


Epoch 9:   0%|          | 0/3532 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.17261918768680118
Validation loss: 0.7387604320913317
F1 Score (Weighted): 0.8111796155906789


Epoch 10:   0%|          | 0/3532 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.15597260465485804
Validation loss: 0.7615036054483761
F1 Score (Weighted): 0.8112539934347164


In [ ]:
torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Bert_model_files/final_bert_model_80%.model')

In [ ]:
#exporting the trained model file
import pickle

with open ('/content/drive/MyDrive/Colab Notebooks/Bert_model_files/Final_BERT_modified_pkl_file_80%.pkl', 'wb') as f:
  pickle.dump(model, f)

In [ ]:
#model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)